In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
from scipy.stats import chi2

In [2]:
data_df = pd.read_csv("/Users/MAC/Desktop/MIP 2019/Code/mydata.txt", header = 0, sep=',', low_memory=False)
data_df = data_df.drop_duplicates()

In [3]:
#visualization between number of tweets and various context types
def visual(df):
    x = df.columns.tolist()[0]
    y = df.columns.tolist()[1]
    plt.xlabel(x)
    plt.ylabel(y)
    plt.bar(np.arange(len(df[x])),df[y])
    plt.xticks(np.arange(len(df[x])), df[x])
    plt.show() 

#test of independence to find context attributes that are statistically significant
def chi2test(df):
    print(df.head())
    print()
    #run Pearson's Chi-Square Test
    stat, p, dof, expected = chi2_contingency(df)
    print('degree of freedom = %d' % dof)
    # interpret test-statistic
    prob = 0.99
    critical = chi2.ppf(prob, dof)
    print('probability = %.3f, critical value = %.3f, statistics = %.3f' % (prob, critical, stat))
    if abs(stat) >= critical:
        print('=> Two variables are dependent')
    else:
        print('=> Two variables are independent')
    # interpret p-value
    alpha = 1.0 - prob
    print('significance level = %.3f, p-value = %.3f' % (alpha, p))
    if p <= alpha:
        print('=> Two variables are dependent')
    else:
        print('=> Two variables are independent')

#filter out an attribute with specific value, eg. female artists        

def filter(attribute, value):
    is_value = data_df[attribute]==value #bool value
    filtered_df = data_df[is_value]
    return filtered_df

In [4]:
### DEALING WITH CONTEXTUAL INFORMATION ###

#creat a dataframe of 2 columns: a parameter and number of tweets posted on that day in the US
day_df = data_df.groupby('tweet_day')['tweet_id'].count().reset_index(name='tweet_count')
month_df = data_df.groupby('tweet_month')['tweet_id'].count().reset_index(name='tweet_count')
time_df = data_df.groupby('tweet_time')['tweet_id'].count().reset_index(name='tweet_count')
song_df = data_df.groupby('track_id')['tweet_id'].count().reset_index(name='tweet_count').sort_values(by = 'tweet_count', ascending=False)
user_df = data_df.groupby('user_id')['tweet_id'].count().reset_index(name='tweet_count')

#create context matrices
timebased_df = data_df.groupby(['user_id','tweet_time'])['tweet_id'].count().unstack(fill_value=0)
daybased_df = data_df.groupby(['user_id','tweet_day'])['tweet_id'].count().unstack(fill_value=0)
daybased_df['weekday'] = daybased_df['weekday'] / 5
daybased_df['weekend'] = daybased_df['weekend'] / 2
monthbased_df = data_df.groupby(['user_id','tweet_month'])['tweet_id'].count().unstack(fill_value=0)